In [ ]:
#%pip install gradio

In [1]:
from datetime import datetime
import gradio as gr
import os
from nClient import HttpClient, HttpResponseUtils, build_request

def client_request(host, port, method, path, save_response, save_filename, headers, upload_file, manual_body, upload):
    host = host.strip() or "localhost"
    port = port.strip()
    port = int(port) if port.isdigit() else 80
    base_path = path.strip() #! Revisar si se puede quitar

    for prefix in ["http://", "https://"]:
        if host.startswith(prefix):
            host = host[len(prefix):]
            break

    if "/" in host:
        host = host.split("/", 1)[0]

    method = method.upper().strip()
    if method not in ["GET", "HEAD", "POST", "PUT", "DELETE"]:
        return "Invalid method. Please enter GET, HEAD, POST, PUT, or DELETE."

    custom_headers = []
    if headers:
        if isinstance(headers, str):
            custom_headers = [h for h in headers.strip().splitlines() if h.strip()]
        elif isinstance(headers, list):
            custom_headers = headers

    body = None
    is_binary = False
    content_type = None
    boundary = None

    if method == 'GET':
        ext = path.split('.')[-1].lower() if '.' in path else ''
        is_binary = ext in ['png', 'jpg', 'jpeg', 'gif', 'mp3', 'wav', 'mp4', 'avi']

    skip_file = False
    if method in ["POST","PUT"] and path.startswith("/resources"):
        parts = path.strip("/").split("/")
        if method == "POST":
            if len(parts) == 1:
                return "For POST to /resources, specify the category in the path."
            elif len(parts) == 2:
                pass
            else:
                return "Invalid POST path. Format: /resources/{category}"
        elif method == "PUT":
            if len(parts) < 4:
                if len(parts) == 1:
                    return "For PUT to /resources, specify the category in the path."
                elif len(parts) == 3:
                    pass
            else:
                return "Invalid PUT path. Format: /resources/{category}/{id}"
        body = manual_body
        content_type = "application/json"
        skip_file = True

    if method in ["POST","PUT"] and not skip_file:
        if upload:
            try:
                file_path = upload_file
                ext = file_path.lower().split('.')[-1]
                is_binary = ext in ['png','jpg','jpeg','gif','mp3','wav','mp4','avi']
                content_type = 'application/octet-stream'
                if ext in ['jpg','jpeg']:
                    content_type = 'image/jpeg'
                elif ext == 'png': content_type = 'image/png'
                elif ext == 'gif': content_type = 'image/gif'
                elif ext == 'mp3': content_type = 'audio/mpeg'
                elif ext == 'wav': content_type = 'audio/wav'
                elif ext in ['txt','html','css','json']:
                    content_type = 'text/plain'
                    is_binary = False
                mode = 'rb' if is_binary else 'r'
                with open(file_path, mode) as f:
                    body = f.read()
            except Exception as e:
                return f"Error reading file: {e}"
        else:
            body = manual_body
            content_type = "application/json"

    request = build_request(
        method=method,
        path=path,
        host=host,
        custom_headers=custom_headers,
        body=body,
        content_type=content_type,
        is_binary=is_binary,
        boundary=boundary
    )

    client = HttpClient(host, port)
    try:
        client.connect()
        response = client.send_request(request, is_binary=is_binary)
        client.close()
    except Exception as e:
        return f"Error: {e}"

    if response:
        headers, content_type, content = HttpResponseUtils.parse_response(response, is_binary)
        if headers:
            result = "\n=== Response Headers ===\n" + headers
            is_binary_content = content_type and any(t in content_type.lower()
                                                     for t in ['image/', 'audio/', 'video/', 'application/octet-stream'])
            if save_response and content:
                if HttpResponseUtils.save_content(content, save_filename, is_binary_content):
                    result += f"\nContent saved to: {save_filename}"
            if content and not is_binary_content:
                result += "\n=== Content Preview ===\n"
                try:
                    result += content.decode() if isinstance(content, bytes) else str(content)
                except:
                    result += "(Binary content)"
            return result
        else:
            return "No response headers."
    return "No response received."


"""
INTERFACE
"""
#! Revisar si se puede quitar upload_file_checked
def update_ui(method, upload_file_checked):
    if method == "GET":
        return (
            gr.update(visible=True),    
            gr.update(visible=False),   
            gr.update(visible=False),   
            gr.update(visible=False),   
            gr.update(visible=False),   
        )
    elif method in ["POST", "PUT"]:
        return (
            gr.update(visible=False),   
            gr.update(visible=False),   
            gr.update(visible=False),   
            gr.update(visible=True),    
            gr.update(visible=True),   
        )
    else:
        return (
            gr.update(visible=False),  
            gr.update(visible=False),  
            gr.update(visible=False),   
            gr.update(visible=False),   
            gr.update(visible=False),   
        )


def toggle_filename_input(save_file_checked):
    return gr.update(visible=save_file_checked)


def toggle_postPut_method(checked):
    if checked:
        return gr.update(visible=True), gr.update(visible=False)
    else:
        return gr.update(visible=False), gr.update(visible=True)


def get_file_path(file):
    return file.name if file else ""


with gr.Blocks() as iface:
    with gr.Row():
        host = gr.Textbox(label="Host", value="localhost")
        port = gr.Textbox(label="Port", value="8080")

    method = gr.Radio(
        choices=["GET", "POST", "PUT", "DELETE", "HEAD"],
        label="HTTP Method",
        value="GET"
    )

    path = gr.Textbox(label="Path", value="/")

    save_file = gr.Checkbox(label="Save file (for GET requests)", visible=True)
    file_name = gr.Textbox(label="Filename to save response", visible=False)
    save_file.change(fn=toggle_filename_input, inputs=save_file, outputs=file_name)

    headers = gr.Textbox(
        label="Custom Headers (one per line)",
        lines=3,
        placeholder="X-Custom: value"
    )

    upload_file_checkbox = gr.Checkbox(
        label="Upload File",
        value=False,
        visible=False
    )

    manual_body = gr.Textbox(
        label="Manual Body",
        lines=5,
        placeholder="Enter request body here...",
        visible=False
    )

    upload_file = gr.File(label="Upload File", type="filepath", visible=False)

    file_path_box = gr.Textbox(visible=False)

    upload_file_checkbox.change(
        fn=toggle_postPut_method,
        inputs=[upload_file_checkbox],
        outputs=[upload_file, manual_body]
    )

    upload_file.change(
        fn=get_file_path,
        inputs=upload_file,
        outputs=file_path_box
    )

    method.change(
        update_ui,
        inputs=[method, upload_file_checkbox],
        outputs=[save_file, manual_body, upload_file, upload_file_checkbox, manual_body]
    )

    response = gr.Textbox(label="Response", lines=10)
    submit = gr.Button("Send Request")

    submit.click(
        fn=client_request,
        inputs=[
            host, port, method, path,
            save_file, file_name,
            headers, file_path_box, 
            manual_body, upload_file_checkbox
        ],
        outputs=response
    )


if __name__ == "__main__":
    iface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Connected (plain) to localhost:8080
Connected (plain) to localhost:8080
Connected (plain) to localhost:8080
Connected (plain) to localhost:8080
Connected (plain) to localhost:8080
Connected (plain) to localhost:8080
Keyboard interruption in main thread... closing server.
